In [23]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import Point
import re

In [3]:
trash_haul = pd.read_csv('../data/trash_hauler_report.csv')

In [4]:
trash_haul

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002


In [5]:
print(trash_haul.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 1.7+ MB
None


Filtering for all missing pickups 

In [6]:
all_missed = trash_haul.loc[trash_haul['Description'].str.contains('miss', case=False, na=False) |  
    trash_haul['Request'].str.contains('miss', case=False, na=False)]
all_missed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203.0,METRO,9208,21.0,1731459.367,666013.6012
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002


Changing incident addresses to all uppercase

In [7]:
all_missed['Incident Address'] = all_missed['Incident Address'].str.upper()

C:\Users\abita\AppData\Local\Temp\ipykernel_35296\929278128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_missed['Incident Address'] = all_missed['Incident Address'].str.upper()


In [8]:
all_missed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 CLARKSVILLE PIKE,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 THUNDERBIRD DR,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 OLD MATTHEWS RD,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 CROLEY DR,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 JO JOHNSTON AVE,37203.0,METRO,9208,21.0,1731459.367,666013.6012
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 MURFREESBORO PIKE,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 LONG AVE, NASHVILLE, TN 37206, UNITED STATES",37206.0,METRO,9508,6.0,1749711.399,669201.6016
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 WINDEMERE CIR, NASHVILLE, TN 37214, UNITE...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 MURFREESBORO PIKE, NASHVILLE, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002


Creating uniform address text

In [9]:
all_missed['Incident Address'] = all_missed['Incident Address'].str.split(',').str[0]
all_missed

C:\Users\abita\AppData\Local\Temp\ipykernel_35296\657079991.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_missed['Incident Address'] = all_missed['Incident Address'].str.split(',').str[0]


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 CLARKSVILLE PIKE,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 THUNDERBIRD DR,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 OLD MATTHEWS RD,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 CROLEY DR,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 JO JOHNSTON AVE,37203.0,METRO,9208,21.0,1731459.367,666013.6012
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 MURFREESBORO PIKE,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,1621 LONG AVE,37206.0,METRO,9508,6.0,1749711.399,669201.6016
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,2943 WINDEMERE CIR,37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,3325 MURFREESBORO PIKE,37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002


Counting duplicate address

In [10]:
address_counts = all_missed['Incident Address'].value_counts()
duplicate_add = address_counts[address_counts >= 2].to_frame().reset_index()
duplicate_add

,Incident Address,count
0,110 GEORGE L DAVIS BLVD,28
1,5135 HICKORY HOLLOW PKWY,23
2,12546 OLD HICKORY BLVD,21
3,6007 OBRIEN AVE,20
4,3710 N NATCHEZ CT,20
...,...,...
3295,4790 DRAKES BRANCH RD,2
3296,3107 CURTIS ST,2
3297,223 BELL RD,2
3298,4311 DAKOTA AVE,2


In [11]:
duplicate_add.columns = ['Incident Address', 'Count']
duplicate_add

,Incident Address,Count
0,110 GEORGE L DAVIS BLVD,28
1,5135 HICKORY HOLLOW PKWY,23
2,12546 OLD HICKORY BLVD,21
3,6007 OBRIEN AVE,20
4,3710 N NATCHEZ CT,20
...,...,...
3295,4790 DRAKES BRANCH RD,2
3296,3107 CURTIS ST,2
3297,223 BELL RD,2
3298,4311 DAKOTA AVE,2


In [12]:
duplicate_sum = duplicate_add['Count'].sum() * 200
duplicate_sum

1966800

Total of all fines for missed pick ups is $1,966,800

In [13]:
prop_damage = trash_haul.loc[trash_haul['Request'].str.contains('damage', case=False, na=False)]
prop_damage

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
6,25325,11/1/2017,Damage to Property,Trash/emptied Wednesday & now metal black-mail...,4721 Chalmers Dr,37215.0,RED RIVER,3303,34.0,1726300.965,637078.4022
173,26971,11/8/2017,Damage to Property,truck is cutting into yard and damaging lawn,717 garrison dr,37207.0,RED RIVER,3205,2.0,1727317.062,688538.0878
257,28158,11/15/2017,Damage to Property,cable lines pulled from house - caused damage ...,6347 alamo pl,37209.0,RED RIVER,4203,20.0,1711130.449,663421.4019
360,29388,11/21/2017,Damage to Property,Customer does not understands why trash-truck ...,717 Troy Dr,37207.0,RED RIVER,3205,2.0,1727518.612,689610.1890
384,29874,11/27/2017,Damage to Property,back door pick up - said truck drove over 2 li...,4627 Fanning Dr,37013.0,RED RIVER,4505,28.0,1769454.565,633049.6026
...,...,...,...,...,...,...,...,...,...,...,...
19969,263831,10/24/2019,Damage to Property,damage to car port when trash was being up pic...,"6922 Highland Park Dr, Nashville, TN 37205, Un...",37205.0,RED RIVER,1303,23.0,1706006.254,636118.1093
20107,265993,10/30/2019,Damage to Property,The driver of our trash collector yesterday hi...,"728 CHERYL AVE, 37115",37115.0,NaN,NaN,9.0,1763161.999,700217.5983
20114,266119,10/30/2019,Damage to Property,Today is trash/pickup unit-A {trash/truck knoc...,"1020 RED BUD TER, 37115",37115.0,RED RIVER,3312,7.0,1759140.999,697993.5988
20143,266348,10/31/2019,Damage to Property,Trash bin was dropped by trash guys,"5225 Tusculum Ct, Nashville, TN 37211, United ...",37211.0,RED RIVER,4404,27.0,1762136.929,626098.2184


In [36]:
all_missed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 CLARKSVILLE PIKE,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 THUNDERBIRD DR,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 OLD MATTHEWS RD,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 CROLEY DR,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 JO JOHNSTON AVE,37203.0,METRO,9208,21.0,1731459.367,666013.6012
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 MURFREESBORO PIKE,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,1621 LONG AVE,37206.0,METRO,9508,6.0,1749711.399,669201.6016
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,2943 WINDEMERE CIR,37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,3325 MURFREESBORO PIKE,37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002
